## 验证模型配置 示例代码

In [1]:
import sys
sys.path.append('../../medlab')
from medlab.registry import MODELS
from mmengine.config import Config
import torch

Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'


In [2]:
import os
print(os.getcwd())

d:\code\QT\med-Ailab\medlab\testing


1. 可以通过指定 ___配置文件___ 的形式， 配置文件中需要包含 ___model___ 字段

In [3]:
# 配置文件路径
config_path = '../medlab/configs/_base_/models/segmentation/attentionunet/attentionunet2d_monai.py'
cfg = Config.fromfile(config_path)
print(cfg)
assert cfg.get('model', None) is not None
model = MODELS.build(cfg.model)
# 生成随机数（注意查看配置文件中的模型输入、输出通道数，部分模型还额外指定输入的尺寸）
x = torch.randn(4, 1, 96, 96)
# 传入模型查看输出
model(x).shape

Config (path: ../medlab/configs/_base_/models/segmentation/attentionunet/attentionunet2d_monai.py): {'model': {'type': 'attentionunet_monai', 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 1, 'channels': (64, 128, 256, 512, 1024), 'strides': (2, 2, 2, 2), 'kernel_size': 3, 'up_kernel_size': 3, 'dropout': 0.0}}


torch.Size([4, 1, 96, 96])

2. 直接传入 ___字典___

In [52]:
cfg = dict(
    type='UNETR',
    in_channels=1,
    out_channels=1,
    img_size=(96, 96),
    feature_size=16, 
    hidden_size=768, 
    mlp_dim=3072, 
    num_heads=12, 
    pos_embed='conv', 
    norm_name='instance', 
    conv_block=True, 
    res_block=True, 
    dropout_rate=0.0, 
    spatial_dims=2, 
    qkv_bias=False
)

In [53]:
model = MODELS.build(cfg)
model

UNETR(
  (vit): ViT(
    (patch_embedding): PatchEmbeddingBlock(
      (patch_embeddings): Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (blocks): ModuleList(
      (0-11): 12 x TransformerBlock(
        (mlp): MLPBlock(
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (linear2): Linear(in_features=3072, out_features=768, bias=True)
          (fn): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (drop2): Dropout(p=0.0, inplace=False)
        )
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SABlock(
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (qkv): Linear(in_features=768, out_features=2304, bias=False)
          (input_rearrange): Rearrange('b h (qkv l d) -> qkv b l h d', qkv=3, l=12)
          (out_rearrange): Rearrange('b h l d -> b l (h d)')
          (drop_output): Dr